In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import signal
from scipy.fft import fft, fftfreq, fftshift
import librosa
import soundfile as sf
import pywt
from scipy.signal import find_peaks, butter, filtfilt
from scipy.ndimage import gaussian_filter
import warnings
import os
import plotly.express as px
import plotly.graph_objects as go
from scipy.io import wavfile
warnings.filterwarnings('ignore')
from plotly.subplots import make_subplots
import networkx as nx

In [3]:
data_file = "../data/scooter_example_1.wav"
fs, data = wavfile.read(data_file)

# crop data
start_time = 80 # seconds
end_time = 150 # seconds
data = data[int(start_time*fs):int(end_time*fs)]

# data_file = "../data/12062025_example.wav"
# fs, data = wavfile.read(data_file)

In [8]:
# Spectrogram parameters
nperseg=65536
hop=0.5
noverlap=int(nperseg * (1 - hop))
window='hann'
title='Spectrogram'
colorscale='Viridis'
crop_freq=2000

# Compute spectrogram
frequencies, times, Sxx = signal.spectrogram(data, fs=fs, window=window, nperseg=nperseg, noverlap=noverlap)

# Crop frequencies if specified
if crop_freq is not None:
    freq_mask = frequencies <= crop_freq
    frequencies = frequencies[freq_mask]
    Sxx = Sxx[freq_mask, :]

# Convert to dB scale
Sxx_db = 10 * np.log10(Sxx + 1e-10)  # Add small value to avoid log(0)
# Sxx_db = np.clip(Sxx_db, a_min=-50, a_max=50)

# Create the heatmap
fig = go.Figure(data=go.Heatmap(z=Sxx_db, x=times, y=frequencies, colorscale=colorscale, colorbar=dict(title='Power (dB)')))

# Update layout
fig.update_layout(title=title, xaxis_title='Time (s)', yaxis_title='Frequency (Hz)', width=800, height=600)
fig.show()

In [9]:
def spectral_visibility_graph(spectrum):
    """
    Build a visibility graph from a 1D magnitude spectrum.
    Returns a NetworkX graph and the degree sequence.
    """
    n = len(spectrum)
    G = nx.Graph()
    G.add_nodes_from(range(n))

    for i in range(n):
        for j in range(i+1, n):
            # check visibility condition
            visible = True
            for k in range(i+1, j):
                if spectrum[k] >= spectrum[i] + (spectrum[j]-spectrum[i]) * (k-i)/(j-i):
                    visible = False
                    break
            if visible:
                G.add_edge(i, j)
    degree_seq = np.array([d for _, d in G.degree()])
    return G, degree_seq


In [10]:
_t = 20  # seconds
ix = np.argmin(np.abs(times - _t))
frame = Sxx_db[:, ix]  # Take the time frame closest to _t
G, degree_seq = spectral_visibility_graph(frame)

In [12]:
# plot degree_seq with plotly
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(len(degree_seq)), y=degree_seq, mode='lines+markers'))
fig.update_layout(title=f'Degree Sequence of Visibility Graph at t={_t}s', xaxis_title='Frequency Bin', yaxis_title='Degree')
fig.show()

In [15]:
# plot visibility graph
pos = nx.spring_layout(G)
edge_x = []
edge_y = []
for edge in G.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_x.append(x0)
    edge_x.append(x1)
    edge_x.append(None)
    edge_y.append(y0)
    edge_y.append(y1)
    edge_y.append(None)
edge_trace = go.Scatter(x=edge_x, y=edge_y, line=dict(width=0.5, color='#888'), hoverinfo='none', mode='lines')
node_x = []
node_y = []
for node in G.nodes():  
    x, y = pos[node]
    node_x.append(x)
    node_y.append(y)
node_trace = go.Scatter(x=node_x, y=node_y, mode='markers', hoverinfo='text', marker=dict(showscale=True, colorscale='YlGnBu', size=10, color=[], colorbar=dict(thickness=15, title='Node Connections', xanchor='left')))
node_trace.marker.color = degree_seq
node_text = [f'Freq Bin {i}<br>Degree {degree_seq[i]}' for i in G.nodes()]
node_trace.text = node_text
fig = go.Figure(data=[edge_trace, node_trace], layout=go.Layout(title=f'Visibility Graph at t={_t}s', showlegend=False, hovermode='closest', margin=dict(b=20,l=5,r=5,t=40), xaxis=dict(showgrid=False, zeroline=False, showticklabels=False), yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)))
fig.show()

In [18]:
_t = 20  # seconds
ix = np.argmin(np.abs(times - _t))

# visibility graph spectrogram for multiple times
S = Sxx_db[:, :ix]  # Downsample in time for faster computation
vgs = np.empty(S.shape, dtype=int)
for i in range(S.shape[1]):
    _, degree_seq = spectral_visibility_graph(S[:, i])
    vgs[:, i] = degree_seq

# plot visibility graph spectrogram
fig = go.Figure(data=go.Heatmap(z=vgs, x=times[::10], y=frequencies, colorscale=colorscale, colorbar=dict(title='Degree')))
fig.update_layout(title='Visibility Graph Spectrogram', xaxis_title='Time (s)', yaxis_title='Frequency (Hz)', width=800, height=600)
fig.show()